<a href="https://colab.research.google.com/github/AayushKaul-ai/NLP_Sentiments/blob/main/DL_5_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 import pandas as pd

In [3]:
df=pd.read_csv('Restaurant_Reviews.tsv',sep='\t')

In [ ]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
# Data Preprocessing
# Lower Case
df['Review']=df['Review'].str.lower()

In [5]:
df.head()

,Review,Liked
0,wow... loved this place.,1
1,crust is not good.,0
2,not tasty and the texture was just nasty.,0
3,stopped by during the late may bank holiday of...,1
4,the selection on the menu was great and so wer...,1


In [6]:
# Removing punctuations
import string
string.punctuation
exclude=string.punctuation

In [7]:
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def remove_punc(x):
  for i in exclude:
    x=x.replace(i,'')
  return x

In [9]:
# example
remove_punc('hey !!, how are you??')

'hey  how are you'

In [10]:
df['Review']=df['Review'].apply(remove_punc)

In [11]:
df.head()

,Review,Liked
0,wow loved this place,1
1,crust is not good,0
2,not tasty and the texture was just nasty,0
3,stopped by during the late may bank holiday of...,1
4,the selection on the menu was great and so wer...,1


In [12]:
# Apply TF-IDFvectorizer on preprocessed data
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
tf=TfidfVectorizer()
x=tf.fit_transform(df['Review'])
x

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9754 stored elements and shape (1000, 2067)>

In [14]:
x.shape

(1000, 2067)

In [15]:
y=df['Liked']

In [16]:
y.head()

,Liked
0,1
1,0
2,0
3,1
4,1


In [17]:
y.shape

(1000,)

In [18]:
# Train test split
from sklearn. model_selection import train_test_split

In [19]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=21)

In [21]:
xtrain.shape

(800, 2067)

In [22]:
# Build Neural Network
from keras.models import Sequential
from keras.layers import Input,Dense,Dropout
from keras.callbacks import EarlyStopping

In [23]:
model=Sequential()

# Add on input layer
model.add(Input(shape=(x.shape[1],)))

# Create a dense layer 1
model.add(Dense(units= 128,activation='relu'))

# Dropout Layer
model.add(Dropout(rate=0.5))

# Add dense layer2
model.add(Dense(units=64,activation='relu'))

# add dropout layer 2
model.add(Dropout(rate=0.4))

# Add output layer
model.add(Dense(units=1,activation='sigmoid'))

In [24]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
es=EarlyStopping(monitor='val_loss',patience=15)

In [26]:
hist=model.fit(xtrain,ytrain,validation_split=0.1,epochs=200,callbacks=[es])

Epoch 1/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5203 - loss: 0.6913 - val_accuracy: 0.3625 - val_loss: 0.6973
Epoch 2/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 30s 859ms/step - accuracy: 0.5878 - loss: 0.6798 - val_accuracy: 0.3750 - val_loss: 0.6987
Epoch 3/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 20s 800ms/step - accuracy: 0.6389 - loss: 0.6546 - val_accuracy: 0.4375 - val_loss: 0.6840
Epoch 4/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 23s 998ms/step - accuracy: 0.7918 - loss: 0.5834 - val_accuracy: 0.6500 - val_loss: 0.6138
Epoch 5/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 30s 425ms/step - accuracy: 0.9100 - loss: 0.4405 - val_accuracy: 0.7875 - val_loss: 0.5081
Epoch 6/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 27s 779ms/step - accuracy: 0.9637 - loss: 0.2465 - val_accuracy: 0.7750 - val_loss: 0.4716
Epoch 7/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 13s 452ms/step - accuracy: 0.9849 - loss: 0.1390 - val_accuracy: 0.7875 - val_loss: 0.4400
Epoch 8/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 453ms/step - accuracy: 0.9869 - loss: 0.0786 - val

In [27]:
# predictions
yprob_train=model.predict(xtrain)
yprob_test=model.predict(xtest)

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step


In [28]:
yprob_train[:5]

array([[9.9915349e-01],
       [9.9996507e-01],
       [9.9999988e-01],
       [5.3779750e-05],
       [3.8942602e-04]], dtype=float32)

In [29]:
ypred_tr=[]
for i in yprob_train:
  if i>0.5:
    ypred_tr.append(1)
  else:
    ypred_tr.append(0)

In [30]:
ypred_tr[:5]

[1, 1, 1, 0, 0]

In [31]:
yprob_test[:5]

array([[3.6525805e-02],
       [2.0463760e-04],
       [9.9281186e-01],
       [9.9995625e-01],
       [9.9999380e-01]], dtype=float32)

In [32]:
ypred_ts=[]
for i in yprob_test:
  if i>0.5:
    ypred_ts.append(1)
  else:
    ypred_ts.append(0)

In [33]:
ypred_ts[:5]

[0, 0, 1, 1, 1]

In [34]:
# CFD
from sklearn.metrics import confusion_matrix,classification_report

In [41]:
st=input('enter your review')
stl=st.lower()
strm=remove_punc(stl)
sttf=tf.transform([strm]).toarray()

enter your reviewIt ws bad


In [42]:
pred=model.predict([sttf])

if pred>0.5:
  print('positive review')
else:
  print('negative review')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
negative review
